<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 3:** LangChain Expression Language</font>

<br>

In the previous notebook, we introduced some of the services we'll be taking advantage of for our LLM applications, including some external LLM platforms and a locally-hosted front-end service. Both of these components incorporate LangChain, but it hasn't been spotlighted as a major focus yet. It is expected that you have some experience with LangChain and LLMs, but this notebook is intended catch you up to the level necessary to progress through the rest of the course!

This notebook is designed to guide you through the integration and application of LangChain, a leading orchestration library for Large Language Models (LLMs), with the AI Foundation Endpoints from last time. Whether you are a seasoned developer or new to LLMs, this course will enhance your understanding and skills in building sophisticated LLM applications.

<br>

### **Learning Objectives:**

- Learning how to leverage chains and runnables to orchestrate interesting LLM systems.  
- Getting familiar with using LLMs for external conversation and internal reasoning.
- Be able to start up and run a simple Gradio interface inside your notebook.

<br>

### **Questions To Think About:**

- What kinds of utilities might be necessary to keep information flowing through the pipeline **(primer for next notebook)**.
- When you encounter the `gradio` interface, consider where all you have seen this style of interface before. Some possible places may include [HuggingFace Spaces](https://huggingface.co/spaces), but did you know that the NGC playground environments for foundation models use it? Hypothesize what kinds of options `gradio` might allow to make this level of flexibility possible?
- Near the end of the section, you'll learn that you can pass around chains as routes and access them across environments via ports. What kinds of requirements should you advertise if you are trying to recieve chains from other microservices?

<br>

### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://learn.next.courses.nvidia.com/courses/course-v1:DLI+S-FX-15+V1/about). If sharing this material, please give credit and link back to the original course.

<br>


### **Environment Setup:**

In [1]:
## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio typing_extensions>=4.8.0

## If you're in colab and encounter a typing-extensions issue,
##  restart your runtime and try again
from langchain_nvidia_ai_endpoints._common import NVEModel

In [2]:
from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try: 
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

NVIDIA API Key:  ········


Retrieved NVIDIA_API_KEY beginning with "nvapi-cVJ..."


{'playground_smaug_72b': '008cff6d-4f4c-4514-b61e-bcfad6ba52a7',
 'playground_llama2_70b': '0e349b44-440a-44e1-93e9-abe8dcb27158',
 'ai-embed-qa-4': '09c64e32-2b65-4892-a285-2f585408d118',
 'playground_gemma_7b': '1361fa56-61d7-4a12-af32-69a3825746fa',
 'ai-rerank-qa-mistral-4b': '0bf77f50-5c35-4488-8e7a-f49bb1974af6',
 'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'playground_yi_34b': '347fa3f3-d675-432c-b844-669ef8ee53df',
 'ai-llama2-70b': '2fddadfb-7e76-4c8a-9b82-f7d3fab94471',
 'playground_mistral_7b': '35ec3354-2681-4d0e-a8dd-80325dcf7c63',
 'playground_llama2_code_70b': '2ae529dc-f728-4a46-9b8d-2697213666d8',
 'playground_kosmos_2': '0bcd1a8c-451f-4b12-b7f0-64b4781190d1',
 'ai-microsoft-kosmos-2': '6018fed7-f227-48dc-99bc-3fd4264d5037',
 'playground_gemma_2b': '5bde8f6f-7e83-4413-a0f2-7b97be33988e',
 'playground_mamba_chat': '381be320-4721-4664-bd75-58f8783b43c7',
 'playground_deplot': '3bc390c7-eeec-40f7-a64d-0c6a719985f7',
 'ai-parakeet-ctc-riva': '2216

<br>

### **Considering Your Models**

Going back to the [**NGC Catalog**](https://catalog.ngc.nvidia.com/ai-foundation-models), we'll be able to find a selection of interesting models that you can invoke from your environment. These models are all there because there is valid use for them in production pipelines, so it's a good idea to look around and find out which models are best for your use cases.

**The code provided includes some models already listed, but you may want (or need) to upgrade to other models if you notice a strictly-better option or a model is no longer available. *This comment will apply throughout the rest of the course, so keep that in mind!***

----

<br>

## **Part 1:** What Is LangChain?

LangChain is an popular LLM orchestration library to help set up systems that have one or more LLM components. The library is, for better or worse, extremely popular and changes rapidly based on new developments in the field, meaning that somebody can have a lot of experience in some parts of LangChain while having little-to-no familiarity with other parts (either because there are just so many different features or the area is new and the features have only recently been implemented).

This notebook will be using the **LangChain Expression Language (LCEL)** to ramp up from basic chain specification to more advanced dialog management practices, so hopefully the journey will be enjoyable and even seasoned LangChain developers might learn something new!

<!-- > <img style="max-width: 400px;" src="imgs/langchain-diagram.png" /> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/langchain-diagram.png" width=400px/>
<!-- > <img src="https://drive.google.com/uc?export=view&id=1NS7dmLf5ql04o5CyPZnd1gnXXgO8-jbR" width=400px/> -->

----

<br>

## **Part 2:** Chains and Runnables

When exploring a new library, it's important to note what are the core systems of the library and how are they used.

In LangChain, the main building block *used to be* the classic **Chain**: a small module of functionality that does something specific and can be linked up with other chains to make a system. So for all intents and purposes, it is a "building-block system" abstraction where the building blocks are easy to create, have consistent methods (`invoke`, `generate`, `stream`, etc), and can be linked up to work together as a system. Some example legacy chains include `LLMChain`, `ConversationChain`, `TransformationChain`, `SequentialChain`, etc.

More recently, a new recommended specification has emerged that is significantly easier to work with and extremely compact, the **LangChain Expression Language (LCEL)**. This new format relies on a different kind of primitive - a **Runnable** - which is simply an object that wraps a function. Allow dictionaries to be implicitly converted to Runnables and let a **pipe |** operator create a Runnable that passes data from the left to the right (i.e. `fn1 | fn2` is a Runnable), and you have a simple way to specify complex logic!

Here are some very representative example Runnables, created via the `RunnableLambda` class:

In [ ]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from functools import partial

################################################################################
## Very simple "take input and return it"
identity = RunnableLambda(lambda x: x)  ## Or RunnablePassthrough works

################################################################################
## Given an arbitrary function, you can make a runnable with it
def print_and_return(x, preface=""):
    print(f"{preface}{x}")
    return x

rprint0 = RunnableLambda(print_and_return)

################################################################################
## You can also pre-fill some of values using functools.partial
rprint1 = RunnableLambda(partial(print_and_return, preface="1: "))

################################################################################
## And you can use the same idea to make your own custom Runnable generator
def RPrint(preface=""):
    return RunnableLambda(partial(print_and_return, preface=preface))

################################################################################
## Chaining two runnables
chain1 = identity | rprint0
chain1.invoke("Hello World!")
print()

################################################################################
## Chaining that one in as well
output = (
    chain1
    | rprint1
    | RPrint("2: ")
).invoke("Welcome Home!")

print("\nOutput:", output)

----

<br>

## **Part 3:** Dictionary Pipelines with Chat Models

There's a lot you can do with runnables, but it's important to formalize some best practices based on use case and current syntax. At the moment, it's easiest to use *dictionaries* as our default variable containers for a few key reasons" 

**Passing dictionaries helps us keep track of our variables by name.**

Since dictionaries allow us to propagate named variables (values referenced by keys), using them is great for locking in our chain components' outputs and expectations.

**LangChain prompts expect dictionaries of values.**

It's quite intuitive to specify an LLM Chain in LCEL to take in a dictionary and produce a string, and equally easy to raise said string back up to be a dictionary. This is very intentional and is partially due to the above reason. 

<br>

### **Example 1:** A Simple LLM Chain

One of the most fundamental components of classical LangChain is the `LLMChain` that accepts a **prompt** and an **LLM**:

- A prompt, usually retrieved from a call like `PromptTemplate.from_template("string with {key1} and {key2}")`, specifies a template for creating a string as output. A dictionary `{"key1" : 1, "key2" : 2}` could be passed in to get the output `"string with 1 and 2"`.
    - For chat models like `ChatNVIDIA`, you would use `ChatPromptTemplate.from_messages` instead.
- An LLM takes in a string and returns a generated string.
    - Chat models like `ChatNVIDIA` deal in messages instead, but it's the same idea! Using an **StrOutputParser** at the end will extract the content from the message.

The following is a lightweight example of a simple chat chain as described above. All it does is take in an input dictionary and use it fill in a system message to specify the overall meta-objective and a user input to specify query the model.

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## Simple Chat Pipeline
chat_llm = ChatNVIDIA(model="llama2_13b")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Only respond in rhymes"),
    ("user", "{input}")
])

rhyme_chain = prompt | chat_llm | StrOutputParser()

print(rhyme_chain.invoke({"input" : "Tell me about birds!"}))

<br>

In addition to just using the code command as-is, we can try using a [gradio interface](https://www.gradio.app/guides/creating-a-chatbot-fast) to play around with our model. Gradio is a popular tool that provides simple building blocks for creating custom generative AI interfaces! The below example shows how you can make an easy gradio chat interface with this particular example chain:

In [ ]:
import gradio as gr

#######################################################
## Non-streaming Interface like that shown above

# def rhyme_chat(message, history):
#     return rhyme_chain.invoke({"input" : message})

# gr.ChatInterface(rhyme_chat).launch()

#######################################################
## Streaming Interface

def rhyme_chat_stream(message, history):
    ## This is a generator function, where each call will yield the next entry
    buffer = ""
    for token in rhyme_chain.stream({"input" : message}):
        buffer += token
        yield buffer

## Uncomment when you're ready to try this. IF USING COLAB: Share=False is faster
gr.ChatInterface(rhyme_chat_stream).queue().launch(share=True, debug=True) 

## NOTE: When you're done, please click the Square button (twice to be safe) to stop the session.

<br>

### **Example 2: Internal Response**

Sometimes, you also want to have some quick reasoning that goes on behind the scenes before your response actually comes out to the user. When performing this task, you need a model with a strong instruction-following prior assumption built-in.

The following is an example "zero-shot classification" pipeline which will try to categorize a sentence into one of a couple of classes.

**In order, this zero-shot classification chain:**
- Takes in a dictionary with two required keys, `input` and `options`.
- Passes it through the zero-shot prompt to get the input to our LLM.
- Passes that string to the model to get the result.

**Task:** Pick out several models that you think would be good for this kind of task and see how well they perform! Specifically:
- **Try to find models that are predictable across multiple examples.** If the format is always easy to parse and extremely predictable, then the model is probably ok.
- **Try to find models that are also fast!** This is important because internal reasoning generally happens behind the hood before the external response gets generated. Thereby, it is a blocking process which can slow down start of "user-facing" generation, making your system feel sluggish.

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## TODO: Try out some more models and see if there are better options
instruct_llm = ChatNVIDIA(model="llama2_13b")

## Zero-shot classification prompt and chain
zsc_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Pick the most likely topic of the sentence. Choose an one option of the following: {options}. Only one word-answers"
    )),
    ("user", "[Options : {options}] {input} = ")
])

zsc_chain = zsc_prompt | instruct_llm | StrOutputParser()

def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options})

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

<br>

### **Example 3: Multi-Component Chains**

The previous example showed how we can coerce a dictionary into a string by passing it through a prompt->LLM chain, so that's one easy structure to motivate the container choice. But is it just as easy to convert the string output back up to a dictionary?

**Yes, it is!** The simplest way is actually to use the LCEL *"implicit runnable"* syntax, which allows you to use a dictionary of functions (including chains) as a runnable that runs each function and maps the value to the key in the output dictionary.

The following is an example which exercises these utilities while also providing a few extra tools you may find useful in practice.

In [ ]:
################################################################################
## Example of dictionary enforcement methods
def make_dictionary(v, key):
    if isinstance(v, dict):
        return v
    return {key : v}

def RInput(key='input'):
    '''Coercing method to mold a value (i.e. string) to in-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))

def ROutput(key='output'):
    '''Coercing method to mold a value (i.e. string) to out-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))

################################################################################
## Common LCEL utility for pulling values from dictionaries
from operator import itemgetter

up_and_down = (
    RPrint("A: ")
    ## Custom ensure-dictionary process
    | RInput()
    | RPrint("B: ")
    ## Pull-values-from-dictionary utility
    | itemgetter("input")
    | RPrint("C: ")
    ## Anything-in Dictionary-out implicit map
    | {
        'word1' : (lambda x : x.split()[0]),
        'word2' : (lambda x : x.split()[1]),
        'words' : (lambda x: x),  ## <- == to RunnablePassthrough()
    }
    | RPrint("D: ")
    | itemgetter("word1")
    | RPrint("E: ")
    ## Anything-in anything-out lambda application
    | RunnableLambda(lambda x: x.upper())
    | RPrint("F: ")
    ## Custom ensure-dictionary process
    | ROutput()
)

up_and_down.invoke({"input" : "Hello World"})

In [ ]:
## NOTE how the dictionary enforcement methods make it easy to make the following syntax equivalent
up_and_down.invoke("Hello World")

----

<br>

## **Part 4: [Exercise]** Rhyme Re-themer Chatbot

Below is a poetry generation example that showcases how you might organize two different tasks under the guise of a single agent. The system calls back to the simple Gradio example, but extends it with some boiler-plate responses and logic behind the scenes.

It's primary feature is as follows:
- On the first response, it will generate a poem based on your response.
- On subsequent responses, it will keep the format and structure of your original rhyme while modifying the topic of the poem.

**Problem:** At present, the system should function just fine for the first part, but the second part is not yet implemented.

**Objective:** Implement the rest of the `rhyme_chat2_stream` method such that the agent is able to function normally.

To make the gradio component easier to reason with, a simplified `queue_fake_streaming_gradio` method is provided that will simulate the gradio chat event loop with the standard Python `input` method

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from copy import deepcopy

inst_llm = ChatNVIDIA(model="mixtral_8x7b")  ## Feel free to change the models

prompt1 = ChatPromptTemplate.from_messages([
    ("system", "Only respond in rhymes"),
    ("user", "{input}")
])
prompt2 =  ChatPromptTemplate.from_messages([
    ("system", (
        "Only responding in rhyme, change the topic of the input poem to be about {topic}!"
        " Make it happy! Try to keep the same sentence structure, but make sure it's easy to recite!"
        " Try not to rhyme a word with itself."
    )),
    ("user", "{input}")
])

## These are the main chains, constructed here as modules of functionality.
chain1 = prompt1 | inst_llm | StrOutputParser()  ## only expects input
chain2 = prompt2 | inst_llm | StrOutputParser()  ## expects both input and topic

################################################################################
## SUMMARY OF TASK: chain1 currently gets invoked for the first input.
##  Please invoke chain2 for subsequent invocations.

def rhyme_chat2_stream(message, history, return_buffer=True):
    '''This is a generator function, where each call will yield the next entry'''

    first_poem = None
    for entry in history:
        if entry[0] and entry[1]:
            ## If a generation occurred as a direct result of a user input,
            ##  keep that response (the first poem generated) and break out
            first_poem = entry[1]
            break

    if first_poem is None:
        ## First Case: There is no initial poem generated. Better make one up!

        buffer = "Oh! I can make a wonderful poem about that! Let me think!\n\n"
        yield buffer

        ## iterate over stream generator for first generation
        inst_out = ""
        chat_gen = chain1.stream({"input" : message})
        for token in chat_gen:
            inst_out += token
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nNow let me rewrite it with a different focus! What should the new focus be?"
        buffer += passage
        yield buffer if return_buffer else passage

    else:
        ## Subsequent Cases: There is a poem to start with. Generate a similar one with a new topic!

        buffer = f"Sure! Here you go!\n\n"
        yield buffer

        return  ## <- TODO: Early termination for generators. Comment this out

        ########################################################################
        ## TODO: Invoke the second chain to generate the new rhymes.

        ## iterate over stream generator for second generation

        ## END TODO
        ########################################################################

        passage = "\n\nThis is fun! Give me another topic!"
        buffer += passage
        yield buffer if return_buffer else passage

################################################################################
## Below: This is a small-scale simulation of the gradio routine.

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=5):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
history = [[None, "Let me help you make a poem! What would you like for me to write?"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = rhyme_chat2_stream,
    history = history
)

In [ ]:
## Simple way to initialize history for the ChatInterface
chatbot = gr.Chatbot(value = [[None, "Let me help you make a poem! What would you like for me to write?"]])

## IF USING COLAB: Share=False is faster
gr.ChatInterface(rhyme_chat2_stream, chatbot=chatbot).queue().launch(debug=True, share=True)

----

<br>

## **Part 5: [Exercise]** Using Deeper LangChain Integrations

This exercise that gives you an opportunity to investigate some example code regarding [**LangServe**](https://www.langchain.com/langserve). Specifically, we refer to the [**`frontend`**](frontend) directory as well as the [**`35_langserve.ipynb`**](35_langserve.ipynb) notebook.

This exercise **does** require the course environment, but feel free to come back to it when you're ready. This service will only be necessary for your final submission!

- Visit [**`35_langserve.ipynb`**](35_langserve.ipynb) and run the provided script to start up a server with several active routes.
- Once done, verify that the following [**LangServe `RemoteRunnable`**](https://python.langchain.com/docs/langserve) works. The goal of a [**`RemoteRunnable`**](https://python.langchain.com/docs/langserve) is to make it easy to host a LangChain chain as an API endpoint, so the following is just a test to make sure that it works.
    - If it doesn't work the first time, there may be an order-of-operations issue. Feel free to try and restart the langserve notebook. LangServe is still rather early in its life cycle (v0.0.35) so some issues may be encountered.
- Assuming that your local instance is fine, open a new tab, copy your address up until right before the first "/lab", and try to access the `:8090` port (i.e. `http://<...>.courses.nvidia.com:8090`). This will contain the deployment of the [**`frontend`**](frontend) folder which you can interact with.
- You may also be interested in trying the port extension `:9012/basic_chat/playground/`, but note that most of the features there require custom handlers which are out of scope for this course.

-----
    
**Note:** This strategy for deploying and relying on LangServe APIs within this type of environment is very non-standard and is made specifically to give students some interesting code to look at. More stable configurations are achievable with optimized single-function containers, and can be found at:
- [**The NVIDIA "AI Chatbot with Retrieval Augmented Generation" Technical Brief**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief)
- [**The NVIDIA/GenerativeAIExamples GitHub Repo**](https://github.com/NVIDIA/GenerativeAIExamples/tree/main/RetrievalAugmentedGeneration)

-----

<br>

## **Part 6:** Wrap-Up

The goal of this notebook was to onboard you into the LangChain Expression Language scheme as well as provide exposure to `gradio` and `LangServe` interfaces for serving LLM functionality! There will be more of this in the subsequent notebook, but this notebook pushes towards intermediate and emerging paradigms in LLM agent development.

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Take a few minutes to look over the `frontend` directory for the deployment recipe and underlying functionality.
2. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.
3. Continue on to the next video, which will talk about **LangChain Running State Chain**.
4. After the video, go on to the corresponding notebook on **LangChain Running State Chain**.

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>